# **The Ecosystem Object Definition**

The ecosystem is a virtual environment in a virtual universe in which virtual robots are created by the **_Chief Robot Programmer_** (CRP). The ecosytem has 'rules' which, if the robots do not follow, they can come to a premature demise. It is up to the CRP to programme their robots to follow system rules. A well programmed robot colony can thrive and prosper, if it is poorly programmed it may not perform so well.

There are a number of system indicators which measure the health of the ecosystem. Your Robot world may use up all the ecosystem resources, contribute  to Greenhouse gas emissions and litter the place with waste.

During the lifetime of the universe the ecosystem might evolve. This means the ecosystem rules change. Robots must therefore evolve or adapt to their new environment. The CRPs must be agile programmers.

A major deadline exists on the 17-December-2021. There will be a major conference of the parties or **COP27** meeting to see whether your ecosystem is on track to meet its emissions targets and deliver a sustainable Robot ecosystem which is operating in harmony with its environment by creating a circular economy.


# **Ecosystem Version 𝛃**

**Version 3.2 19-Nov 10:00**

##**/Todo**
Drones can move on the ground!


## **Required Imports**

In [ ]:
#@title Required Imports [code]
from IPython.display import clear_output
from copy import copy, deepcopy
import time
from random import randint
from sys import exit
import numpy as np
import matplotlib.pyplot as plt
import math
import types
from queue import Queue


## **Globals**

Global variables used in the code base. Do not adjust these.

In [ ]:
#@title Globals [code]

default_width = 80
default_height = 40
default_altitude = 5
arena_limits = (default_width,default_height,default_altitude)

#Fading of robots  alpha = m_fade * soc/capacity + c_fade
c_fade = 0.1
m_fade = 0.9 - c_fade

#maximum damage a robot can sustain
max_damage = 5
motion_efficiency = 0.100
system_efficiency = 0.025

NL = "\n"
TB = "\t"
PR = ">"
testing = True


## **Support Functions**

The ecosystem requires all these sections to operate. Do not remove and ensure they are run before developing and running your Robots

In [ ]:
def coin():
  return [-1,1][randint(0,1)]

In [ ]:
# @title Irradiance

def normpdf(x, mean, sd):
    var = float(sd)**2
    denom = (2*math.pi*var)**.5
    num = math.exp(-(float(x)-float(mean))**2/(2*var))
    pdf = num/denom
    return pdf

def irradiance(hour):
    hour = hour % 24
    mean = 12     #mid-day
    sd = 3        #standard dev of irradiance
    normalised_pdf  = normpdf(hour,mean,sd)/normpdf(mean,mean,sd)
    moonlight = 0.2
    a = moonlight + (1-moonlight)*normalised_pdf
    return a

#Test Display of irradiance over 24 hours
x,i = [],[]
for h in range(0,100):
  i.append (irradiance(h/4))
  x.append (h/4)
fig = plt.figure(figsize=(4,4))
ax = fig.add_subplot(111)
a = ax.plot(x,i)
a = ax.fill(x,i,'yellow')
plt.title ('Normalised Irradiance')




### **Robot Defaults**
This section defines default values for different Robot types. CRPs should study this section to understand the different default values assigned to
robots of difference types.

In [ ]:
#@title Validation Lists {display-mode: "code"}

####################
## ROBOT_DEFAULTS ##
####################

statuses = ['off', 'on', 'broken']
activities = ['idle', 'charging', 'collecting', 'delivering', 'loading', 'unloading', 'is_cargo', 'under_repair','repairing']
colors = ['blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'black', 'white']
shapes = ['square', 'circle', 'triangle']


In [ ]:
#@title Validation Functions {display-mode: "code"}
lambda_inlist = lambda nv,v,av: (nv in av, av[0])
lambda_nochange = lambda nv,v,av: (nv == v, v)
lambda_inrange = lambda nv,v,av: (av[0] <= nv <= av[1],v)
lambda_listdelta = lambda nv,v,av:(False not in [abs(i_nv - i_v) <= i_av for i_nv, i_v, i_av in zip(nv,v,av)], v)
lambda_listmaxdelta = lambda nv,v,av:(False not in [abs(i_nv - i_v) <= i_av for i_nv, i_v, i_av in zip(nv,v,av)], [ i_v + [-1,1][i_nv > i_v]*min(i_av,abs(i_nv-i_v)) for i_nv, i_v, i_av in zip(nv,v,av)])
lambda_onarena = lambda coordinates: (False not in [0 <= i_c <= i_l for i_c, i_l in zip(coordinates, arena_limits)], [i_c if 0 <= i_c <= i_l else -1 if i_c < 0 else i_l + 1 for i_c, i_l in zip(coordinates, arena_limits)])

def f_validcoordinates(nv,v,av):
  deltaOK, cnv = lambda_listmaxdelta(nv,v,av)  #corrected new values wrt to vector increments
  on_arena, anv = lambda_onarena(cnv)
  return deltaOK and on_arena, anv
#unit tests
if testing:
  print ('inlist',lambda_inlist('squae',1,shapes))
  print('nochange',lambda_nochange(2,1,0))
  print('inrange',lambda_inrange(50,300,[100,1000]))
  print('listDelta',lambda_listdelta([12,14,1],[11,15,13],[1,1,1]))
  print('listmaxdelta',lambda_listmaxdelta([12,7,1],[10,10,3],[2,2,2]))
  print ('onarena',lambda_onarena([45,45,4]))
  print('f_validcoordinates',f_validcoordinates([81,35,1],[79,34,0],[2,2,0]))


In [ ]:
#@title Robot Defaults [code] {display-mode: "code"}

####################
## ROBOT_DEFAULTS ##
####################

#robot_default returns a dictionary of robot instance variables, and properties 
#for these atributes
#The attribute properties are a list of three items:
# 0 is the compulsory flag for the instance variable
# 1 is the default registration value for the robot
# 2 gives permitted changes to the variable (None means any change is permitted)



def robot_default (kind, mode = 'values'):

  #mode can be default or allowed

  x = randint(1,default_width)
  y = randint(1,default_height)
  z = 0
  default = {}
  default['coordinates'] = [[x, y, z],[1, 1, 0 ], f_validcoordinates, 'list (integers), x, y, z location of a robot' ]
  default['on_arena'] = [1,0, lambda_nochange, 'boolean, flag determines if robot is on the arena (read only)']
  default['status'] = ['off',statuses, lambda_inlist, 'string, (member of statuses) determines if robot is on, off or broken']
  default['activity'] = ['idle', activities, lambda_inlist, 'string, (member of activities), determines activity of robot']
  default['age'] = [0, 0, lambda_nochange,'integer, age of robot in hours - (read only)']
  default['soc'] = [600, 0, lambda_nochange, 'integer, state of charge of battery (read only)']
  default['capacity'] = [600, 0, lambda_nochange, 'integer, energy capacity of robot battery (read only)']
  default['service'] = [0, 0, lambda_nochange, 'integer, service points accrued by robot (read only)']
  default['kind'] = [kind, 0, lambda_nochange, 'string, robot class type (read only)']
  default['shape'] = ['square', shapes, lambda_inlist, 'string, (member of shapes) determines arena display shape']
  default['color'] = ['blue', colors, lambda_inlist, 'string, (member of colors) determines arena display colour']
  default['size'] = [250, [250,1000], lambda_inrange, 'integer, (between 250 and 1000) determines arena display size']
  default['alpha'] = [1, 0, lambda_nochange, 'float, (between 0 and 1) determines arena display transparency (read only)']
  default['weight'] = [50,0, lambda_nochange, 'integer, weight of robot (read only)']
  default['payload'] = [100,0, lambda_nochange, 'integer, maximum load robot can carry (read only)']
  default['cargo'] = [None,object, lambda_nochange, 'object, member object robot is transporting (read only)']
  default['distance'] = [0, 0, lambda_nochange, 'float, total distance travelled by robot (read only)']
  default['energy'] = [0, 0, lambda_nochange, 'float, total energy consumed by robot (read only)']
  default['damage'] = [0, 0, lambda_nochange, 'integer, gives damage points accrued by robot (read only)']
  if kind == 'Droid':
    default['coordinates'][1] = [2, 2, 0 ]  
    default['soc'][0] = 2000
    default['capacity'][0] = 2000
    default['shape'][0]= 'circle'
    default['color'][0] = 'red'
    default['size'][0] = 300
    default['weight'][0] = 100
    default['payload'][0] = 200
  elif kind == 'Drone':
    default['coordinates'][1] = [3, 3, 1]   
    default['soc'][0] = 500      
    default['capacity'][0] = 500
    default['shape'][0] = 'triangle'
    default['color'][0] = 'green'
    default['size'][0] = 500
    default['weight'][0] = 25
    default['payload'][0] = 50

  if mode == 'default':
    return default
  elif mode == 'values':
    return {key: value[0] for key, value in default.items()}
  elif mode == 'allowed':
    return {key: value[1] for key, value in default.items()}
  elif mode == 'lambdas':
    return {key: value[2] for key, value in default.items()}    
  else:
    return tuple(key for key in default)

In [ ]:
#@title Robot Defaults Tests {display-mode: "code"}
local_testing = True
if testing or local_testing:
  for kind in ['Robot','Droid','Drone']:
    for return_type in ['default', 'values', 'allowed']:
      print(kind, return_type,robot_default(kind, return_type))

  print('Lambdas:',robot_default('Robot', 'lambdas'))    
  print ('Keys:',robot_default('Robot','keys'))

In [ ]:
#@title Markdown Table Generators {display-mode: "code"}
if testing:
  w = 15
  w2 = 40
  for kind in ['Robot','Droid','Drone']:
    print ('**Defaults values for \'' + kind +'\'**',NL)
    key = 'variable'
    default = 'default'
    description = 'description'
    print (f"|{key:<{w}}|{default:<{w}}|{description:<{w2}}|")
    print (f"|{'-'*w}|{'-'*w}|")
    values = robot_default(kind, 'default')

    for key, value in values.items():
      default,validation,valfunc,description = value
      print (f"|{key:<{w}}|{str(default):<{w}}|{description:<{w2}}|")
    print (NL)

In [ ]:
#@title Object Generators {display-mode: "code"}

def init(self, name):
  self.name = name
  self.directions = [0, 0, 0]
  self.status = 'off'

def move(self):
  self.coordinates[0] += self.directions[0]
  self.coordinates[1] += self.directions[1]
  self.coordinates[2] += self.directions[2]

def generate_class(kind):
  attributes = robot_default(kind, 'values')
  attributes['__init__'] = init
  attributes['move'] = move
  attributes['kind'] = kind
  Dynamic_Robot = type(kind, (), attributes)
  return Dynamic_Robot



### **Display**

The display is updated everytime the eco system is updated although this can be switch off in order to perform rapid simulations.

You can also call display directly for test purposes which bypasses Ecosystem updating, but you can visualise the results of robot programming without ecosystem constraints!

calling display

>
> display (markers, **kwargs)
>
> arguments:
>
> markers: dictionary of markers in format provided by the ecosystem
>
> kwargs: (keyword arguments)
> >
> > width  - integer,  width of chart in cm default = 30
> > 
> > height - integer, height of chart in cm default = 15
> >
> >pause - integer, delay time in ms after displaying chart default = 250
> >
> >title - string, text displayed on chart default = 'Ecosystem Display'
> >
> >clear - boolean, clears code cell output area prior to display default = True)
> >
> >hour - ecosystem time in hours default = 12
> >




In [ ]:
# @title Display Function {display-mode: "code"} 
#%matplotlib inline

# Matplotlib colors and shapes semantics
mpl_colors = {'blue': 'b', 'green': 'g', 'red': 'r', 'cyan': 'c', 'magenta': 'm', 'yellow': 'y', 'black': 'k', 'white': 'w'}
mpl_shapes = {'square':'s', 'circle':'o','triangle':'^'}


def display(markers,**kwargs):
  cm = 1/2.54
  width = kwargs.get('width', 30)*cm
  height = kwargs.get('height', 15)*cm
  pause = kwargs.get('pause', 250)
  title = kwargs.get('title','Ecosystem Display')
  clear = kwargs.get('clear',True)
  hour =  kwargs.get('hour', 12)

  x_max =  default_width
  y_max =  default_height

  if clear:
    clear_output(wait=True)
  
  a = irradiance(hour)

  fig = plt.figure(figsize=(width,height))
  fig.patch.set_facecolor('grey')
  fig.patch.set_alpha(0.6)
  ax = fig.add_subplot(111)
  ax.patch.set_facecolor('yellow')
  ax.patch.set_alpha(a)

  plt.title (title)
  plt.xlim(-2, x_max+2)
  plt.ylim(-2, y_max+2)

  for p in markers:
    try:
      x=p['coordinates'][0]
      y=p['coordinates'][1]
      scale = p['size']
      color = p['color'] 
      shape = p['shape']
      alpha = p['alpha']
      marker_shape = mpl_shapes[shape] 
      ax.scatter(x, y, s=scale, c=color, marker=marker_shape, alpha=alpha, edgecolors='b')
      if p['coordinates'][2] > 0:
        ax.scatter(x, y, s=scale*3, c='k', marker='1', alpha=alpha, edgecolors='b')
      try:
        plt.annotate(p['soc'], (x+.5, y+.5),)
      except:
        pass


    except Exception as error:
      #print("Point error:", p, error)
      continue
    
  ax.set_xticks(range(0, x_max + 1, 10))
  ax.set_yticks(range(0, y_max + 1, 10))
  ax.grid(color = 'red', linestyle = '--', linewidth = 0.25)

  plt.show()
  time.sleep(pause/1000)



In [ ]:
#@title Display Tests {display-mode: "code"}
if testing:
  points = []
  points.append({'coordinates': [50,20,0], 'size': 500, 'color': 'red', 'shape': 'circle', 'alpha': .9})
  points.append({'coordinates': [20,30,1], 'size': 500, 'color': 'blue', 'shape': 'triangle', 'alpha': .2})
  points.append(robot_default('Robot'))
  points.append(robot_default('Drone'))
  points.append(robot_default('Droid'))
  display(points, title = "Ecosystem Display Test")

## **Ecosystem Class**

The **ecosysystem class** provides an interface with a constructor method, a number of public **methods** and **properties** which are important for the CRP.

**Constructor** 

There are no arguments required nor accepted by the constructor

> ecosystem = Ecosystem()

**Public properies**

> robots - returns a list of all registered robot objects (read only)
>
> register - returns a dictionary which is full register of all registered robots (read only)
>
> duration - returns an integer which is the duration of the ecosystem in hours (default 24) (read/write)
>
> hour - returns an integer for the current hour of operation (read only)
>
> stop - return a boolean true value if hour >= duration (read only)
>
> count_operational - returns an integer which is the count of robot with status = 'on' (read only)
>
> messages  - returns boolean True if the fifo message stack has unread messages, and false if empty (read only)
>
> message - returns a tuple message in the fifo message stack, or false if non available. (read/write)
>
> display_on_update - boolean to determine of the robot arena is auto-refreshed on update
>

**Methods**
>
> display(**kwargs) - displays the robot arena and render the robot objects
>   
>   > arguments: kwargs - see chart
>
> register(robot) - registers a robot to participate in ecosystem activities
>
>   > robot: a valid robot object class/subclass
>
> update(**kwargs) - updates the robot register with new values for all attributes, validates permitted changes and makes necessary updates to system variables. If display_on_update is true the robot arena is updated.
>
>   arguments: kwargs are passed to the display method- see chart
>
>







In [ ]:
########################################################
# @title Ecosystem Class [code] {display-mode: "code"}
########################################################

class Ecosystem:
  def __init__(self):
      #Constructor - called when a new object is created using the class
      #Attributes or properties
      self._duration = 24
      self._hour = 0
      self._robots = []
      self.display_on_update = True
      self._register = {}
      self.markers = []
      self._messages = Queue()

 ####################
 ##   Properties   ##
 ####################

  @property
  def messages(self):
      return not self._messages.empty()   #while not q1.empty():
  @property
  def message(self):
    if self._messages.empty():
      return False
    else:
      return self._messages.get()
  @message.setter
  def message(self, value):
    self._messages.put(value)
  @property
  def robots(self):
      return self._robots
  @property
  def register(self):
      return self._register
  @property
  def duration(self):
      return self._duration
  @duration.setter
  def duration(self, value):
      self._duration = value
  @property
  def hour(self):
      return self._hour
  @property
  def count_operational(self):
    count = 0
    for value in self._register.values():
      if value['status'] == 'on':
        count += 1
    return count

  @property
  def stop(self):
    return self._hour>self._duration

 ####################
 ##   METHODS      ##
 ####################

##   DISPLAY       ##
  def display (self, **kwargs):
    markers = self.get_markers()
    display(markers, **kwargs)

##   MARKERS       ##
  def get_markers(self):
    self.markers = [value for value in self._register.values() ]
    return self.markers
 
 ##   REGISTER  ##
  def register (self, robot):
    if robot in self._robots:
      register = self._register[id(robot)]
      if register['status']  == 'on':
        self.message = (robot.name, 'register', 'warning', 'robot already registered')
      else:
        #Re-registering a 'broken' or 'off' is required 
        register['status']  = 'on'
        try:
          setattr(robot, 'status', 'on')
        except:
          pass
        self.message = (robot.name,'register', 're-registered', "succesful re-boot")
    else:
      #Retrieve default values
      try:
        default = robot_default(type(robot).__name__,'values')
        default['status'] = getattr(robot,'status',default['status'])
        for key, value in default.items():
          try:
            setattr(robot, key, deepcopy(value))   #use copy to keep lists unique
          except:
            #User has not coded to receive updates on unrequired instance variables
            pass
        #We're safe to add the robot and update the registry
        self._robots.append(robot)
        self._register[id(robot)] = deepcopy(default)

        #print (PR*2, robot.status )

      except Exception as err:
        self.message = (type(robot), 'register', 'exception', 'Invalid object type') 
        

## UPDATE ##
  
  def update(self, **kwargs):
    for robot in self._robots:
      #get the current register
      register = self._register[id(robot)]
      #take a deepcopy of the register
      new_register = deepcopy(register)


      if new_register['status'] == 'on':

        #update this with any new values from the robot, ignore if not present
        for key, value in new_register.items():
          new_register[key] = getattr(robot,key,value)

        #Prepare to itereate through robots which are on to update
        damage = 0
        #gather iterables to zip
        keys = register.keys()
        new_values = new_register.values()
        values = register.values()
        allowed_values = robot_default(robot.kind,"allowed").values()   #this is quite inefficient
        lambdas = robot_default(robot.kind,"lambdas").values()          #this is quite inefficient

        for key, new_value, value, allowed_value, validation_lambda in zip(keys, new_values, values, allowed_values, lambdas):
          #correct any object updates which are not allowed and add up penalties
          try:
            #print(PR*3,key,new_value, value, allowed_value, validation_lambda, 'Validation: ', validation_lambda(new_value, value, allowed_value))
            if new_value != value:
              #print(PR*4,key,new_value, value, allowed_value, validation_lambda, 'Validation: ', validation_lambda(new_value, value, allowed_value))
              valid, revert = validation_lambda(new_value, value, allowed_value)
              if not valid:
                self.message = (robot.name, 'update', 'damaged', 'Invalid attempt to change ' + key + ' to ' + str(new_value) + ' from ' + revert )
                damage += 1
                new_register[key] = revert
          except:
            pass

      # increment age regardless of anything
      new_register['age'] += 1

      if new_register['status'] == 'on':

        # add damage causes by illegal use of instance variable
        new_register['damage'] += damage

        distance = round(((new_register['coordinates'][0] - register['coordinates'][0])**2 + (new_register['coordinates'][1] - register['coordinates'][1])**2)**.5,1)
        new_register['distance'] += distance
        
        height = max(new_register['coordinates'][2] - register['coordinates'][2],0)
              
        motion_energy = new_register['weight'] * distance * motion_efficiency
        system_energy = new_register['weight'] * system_efficiency
        total_energy = system_energy + motion_energy

        new_register['soc'] -= total_energy
        new_register['soc'] = round(max(new_register['soc'], 0),2)

        if lambda_onarena(new_register['coordinates'])[0] == False:
          new_register['status'] = 'broken'
          self.message = (robot.name, 'update', 'broken', 'electromagnetic damage at ecosystem boundary') 
        elif new_register['damage'] > max_damage:
          self.message = (robot.name, 'update', 'broken', 'max damage score')
          new_register['status'] = 'broken'
        if new_register['soc'] == 0:
          new_register['status'] = 'broken'
          self.message = (robot.name, 'update', 'broken', 'out of power') 

        if new_register['status'] == 'broken':
          new_register['alpha'] = 0.2
        else:
          #alpha  determines the transparency of robots. Running out of fuel makes robots fade
          new_register['alpha'] = m_fade * new_register['soc']/new_register['capacity'] + c_fade

      # End of status != off block             

      #update the robot with new_register
      for key, value in new_register.items():
        try:
          setattr(robot,key,value)
        except:
          pass
      #update the _register with the new values
      self._register[id(robot)] = deepcopy(new_register)
      #print (new_register['coordinates'], register['coordinates'], "d:",distance,"e:", energy, "s:", new_register['soc'], new_register['capacity'], new_register)
    self._hour += 1  
    if self.display_on_update:
      self.display(**kwargs)



# **Stage 1 - Robot Classes**

**Task**

Begin the design your robot classes here. You may, even must, develop your classes further to improve their functionality as you move through the stages.

There are three classes you need initially. 

* Robot - base or parent class
* Droid - inherits from Robot
* Drone - inherits from Robot

Interface requirements are included below.

**Note** future ecosystem evolutions may require new classes, properties and methods. Details will be included in the upgrade notices with future releases of the ecosystem.

**Robot Classes CRP Notes**

**CRPs should augment code cells with text cells to document designs for third parties and assessment in your final submission. It's good practice to developing working text cells for this purpose from day 1

## **Robot Class Specifications**

This section defines the _interface_ for your Robots.

Robot is the parent class from which all other robot kinds inherit. The robot class ***must*** have the following class variables:
>
> kind  (read only) - value assigned by the ecosystem
>

_kind_ is the class name. Although this accessible from the class method it demonstrates a class variable

And the following instance variables.
>
> [x,y,z] list (read/write) robot coordinates - this is how you move your robot.
>
> status - string (on, off or broken) (read write) - you can change this but the system will also enforce the status if rules are broken.
>
_status_ is a member of the statuses dictionary and is equal to 'on' 'off' or 'broken' Robots with a status anything other than 'on' cannot function

x,y,z coordinates are randomly allocated on registration (see instantiation below).

If these _data members_ do not exist the ecosystem may report an 'inadequate interface' or otherwise fail to respect your robots.

All robots are monitored by the ecosystem every ecosystem.update() call. You can, optionally, and according to your own preceived requirements, enhance the interface of your robots to include any of the following additional instance variables (the essential ones above are also included in the table for completeness):


**Instance Variables for 'Robot'** 

|variable       |default        |description                             |
|---------------|---------------|---|
|coordinates    |[x, y, z]    |list of integers, x, y, z location of a robot|
|on_arena       |1              |boolean, flag determines if robot is on the arena (read only)|
|status         |off            |string, (member of statuses) determines if robot is on, off or broken|
|activity       |idle           |string, (member of activities), determines activity of robot|
|age            |0              |integer, age of robot in hours - (read only)|
|soc            |600            |integer, state of charge of battery (read only)|
|capacity       |600            |integer, energy capacity of robot battery (read only)|
|service        |0              |integer, service points accrued by robot (read only)|
|kind           |Robot          |string, robot class type (read only)    |
|shape          |square         |string, (member of shapes) determines arena display shape|
|color          |blue           |string, (member of colors) determines arena display colour|
|size           |250            |integer, (between 250 and 1000) determines arena display size|
|alpha          |1              |float, (between 0 and 1) determines arena display transparency (read only)|
|weight         |50             |integer, weight of robot (read only)    |
|payload        |100            |integer, maximum load robot can carry (read only)|
|cargo          |None           |object, member object robot is transporting (read only)|
|distance       |0              |float, total distance travelled by robot (read only)|
|energy         |0              |float, total energy consumed by robot (read only)|
|damage         |0              |integer, gives damage points accrued by robot (read only)|


Droids and Drones have the same variable requirements and potential. See the Ecosystem Defauls section for more information on the defaults.

**Methods**

There are no compulsory methods for your Robot classes required by the Ecosystem.





In [ ]:
############################################
# @title Robot Class {display-mode: "code"} 
############################################

#Design your robot class here. 



In [ ]:
############################################
# @title Droid Class {display-mode: "code"} 
############################################

#Design your droid class here. 

In [ ]:
############################################
# @title Drone Class {display-mode: "code"} 
############################################

#Design your drone class here. 

# **Stage 2 - Training and Testing**

Training and testing requirements will be published during week 8.

---



# **Stage 3 - Ecosystem 1 Operations**

# **Stage 4 - Ecosystem 2 Operations**

#**Students Questions**

#**Discussion and Conclusions**